In [1]:
import Comp_classify_def as apply
import re
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import TweetTokenizer, RegexpTokenizer

In [2]:
import pandas as pd
from googlesearch import search
import wordninja
import requests
from bs4 import BeautifulSoup
from sklearn.cross_validation import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier

/home/roshan/anaconda3/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [3]:
#Getting the labeled companies
workdf = pd.read_csv('testdata_Labeled.csv')
workdf.drop([workdf.columns[1],'Unnamed: 3','Unnamed: 4','Unnamed: 5','Unnamed: 6',
            'Unnamed: 7'],axis = 1,inplace = True)
workdf.drop(0,axis = 0,inplace = True)
workdf = workdf.head(73)
#Removing the Categories that are Bank to provide independency.
workdf.drop(workdf[workdf['Category']=='Bank'].index,axis = 0,inplace = True)

In [4]:
#Limited Data but works for now
workdf.shape

(43, 2)

In [5]:
workdf = apply.transform_data(workdf)

In [6]:
def link_obtain(X_df):
    for i,j in X_df.iterrows():
        URL = X_df['housing_flg'][i]
        query = URL
        print(query)
        #Directly Retrieving all ifsc code data without saving link in column.
        if(len(str(query.strip())))<5:
            t =0
            str_list = list('https://www.ifsccodebank.com/search-by-IFSC-code.aspx?IFSCCode=')
            str_list.append(query)
            url = ''.join(str_list)
            response = requests.get(''.join(str_list))
            soup = BeautifulSoup(response.text,'html.parser')
            t = soup.get_text()
            r = t[t.find(query)+7:]
            r = (r[:r.find('-')-5].strip())
            X_df['Company'][i] = r
            q2 = str(X_df['Company'][i])+' wiki'
            print(q2)
            if 'com' not in q2:
                t =0
                for l in search(q2,stop = 2):
                    stri = 'Link'+str(t)
                    print(q2,l)
                    X_df[stri][i] = l
                    t = int(t)+1
                    print(stri)
            else:
                X_df['Company'][i] = 'Nan'
            continue
        #if Name has no space ---> Using wordninja algo to split the words.
        elif ' ' not in query.strip():
            query = " ".join(str(x) for x in wordninja.split(query))+' Wiki'
    ####===> The following gets the link and not the data <===####    
        # Direct search for appropriate official Link    
        try:
            t = 0
            for k in search(query,stop = 2):
                stri = 'Link' + str(t)
                X_df[stri][i] = k
                t = t+1
                print(stri)
        # Search the words with key word Financial Services
            t = 2
            query = URL + ' Finanancial Services' + ' Wiki'
            for k in search(query,stop = 2):
                stri = 'Link' + str(t)
                X_df[stri][i] = k 
                t = t+1
                print(stri)
        except Exception as e:
            print(e)
            # except block to store the company which has bad request/other issue
            X_df['Link4'][i] = query
            print('Errored out: ',query)
    return X_df

In [ ]:
workdf = link_obtain(workdf)

AADHAR HOUSING FINAN
Link0
Link1
Link2
Link3
AAMAADMIPARTY
Link0
Link1
Link2
Link3
Aavas Financiers Lim
Link0
Link1
Link2
Link3
ABHIPRA CAPITAL LTD
Link0
Link1
Link2
Link3
DR FILE DTD 
Link0
Link1
Link2
Link3
DR TPZ DT 
Link0
Link1
Link2
Link3
ACHAL
Link0
Link1


In [ ]:
workdf

In [ ]:
def process_text(text):
    soup = BeautifulSoup(text, "lxml")
    tags_del = soup.get_text()
    no_html = re.sub('<[^>]*>', '', tags_del)
    tokenized = casual_tokenizer(no_html)
    lower = [item.lower() for item in tokenized]
    tagged = nltk.pos_tag(lower)
    lemma = lemma_wordnet(tagged)
    no_num = [re.sub('[0-9]+', '', each) for each in lemma]
    no_punc = [w for w in no_num if w not in punc]
    no_stop = [w for w in no_punc if w not in stop_words]
    return no_stop

In [ ]:
#Function to replace the nltk pos tags with the corresponding wordnet pos tag to use the wordnet lemmatizer
def get_word_net_pos(treebank_tag):
    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return None
    
def lemma_wordnet(tagged_text):
    final = []
    for word, tag in tagged_text:
        wordnet_tag = get_word_net_pos(tag)
        if wordnet_tag is None:
            final.append(lemmatizer.lemmatize(word))
        else:
            final.append(lemmatizer.lemmatize(word, pos=wordnet_tag))
    return final

In [ ]:
def data_desc(X_df):
    for i,j in X_df.iterrows():
            if 'Nan' not in X_df['Link0'][i]:
                response = requests.get(X_df['Link0'][i],verify = False)
                t = process_text(response.text)
                X_df['Data0'][i] = t
            if 'Nan' not in X_df['Link1'][i]:
                response = requests.get(X_df['Link1'][i],verify = False)
                t = process_text(response.text)
                X_df['Data1'][i] = t
            if 'Nan' not in X_df['Link2'][i]:
                response = requests.get(X_df['Link2'][i],verify = False)
                t = process_text(response.text)
                X_df['Data2'][i] = t
            if 'Nan' not in X_df['Link3'][i]:
                response = requests.get(X_df['Link3'][i],verify = False)
                t = process_text(response.text)
                X_df['Data3'][i] = t
    return X_df

In [ ]:
#TO show IMBALANCED CLASSES:
%matplotlib inline
import matplotlib.pyplot as plt
workdf.groupby('Category').housing_flg.count().plot.bar(ylim = 0)
plt.show()

In [ ]:
def casual_tokenizer(text): #Splits words on white spaces (leaves contractions intact) and splits out trailing punctuation
    tokens = tokenizer.tokenize(text)
    return tokens

In [28]:
data_desc(workdf)

/home/roshan/.local/lib/python3.5/site-packages/urllib3/connectionpool.py:851: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


NameError: name 'tokenizer' is not defined